In [ ]:
import sys
# use line-buffering for both stdout and stderr
# sys.stdout = open(sys.stdout.fileno(), mode='w', buffering=1)
# sys.stderr = open(sys.stderr.fileno(), mode='w', buffering=1)

import os
import pathlib
import click
import hydra
import torch
import dill
import wandb
import json
from diffusion_policy.workspace.base_workspace import BaseWorkspace
import h5py

In [2]:
# checkpoint="data/0550-test_mean_score=0.969.ckpt" 
checkpoint = "/root/diffusion_policy/data/outputs/2025.05.12/06.08.55_train_diffusion_unet_lowdim_lift_lowdim/checkpoints/epoch=0050-test_mean_score=1.000.ckpt"
output_dir="data/eval_analysis" 
device="cuda:0"

if os.path.exists(output_dir):
    click.confirm(f"Output path {output_dir} already exists! Overwrite?", abort=True)
pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

# load checkpoint
payload = torch.load(open(checkpoint, 'rb'), pickle_module=dill)
cfg = payload['cfg']
cls = hydra.utils.get_class(cfg._target_)
workspace = cls(cfg, output_dir=output_dir)
workspace: BaseWorkspace
workspace.load_payload(payload, exclude_keys=None, include_keys=None)

In [3]:
# get policy from workspace
policy = workspace.model
if cfg.training.use_ema:
    policy = workspace.ema_model

device = torch.device(device)
policy.to(device)
policy.eval()
pass 

In [ ]:
# run eval
env_runner = hydra.utils.instantiate(
    cfg.task.env_runner,
    output_dir=output_dir)

In [ ]:
runner_log = env_runner.run(policy)
runner_log['test/mean_score']

In [ ]:
kwargs={'epoch':100, 'save_rollout':True}
runner_log = env_runner.run(policy, kwargs)
runner_log['test/mean_score']

In [ ]:
kwargs={'epoch':200}
runner_log = env_runner.run(policy, kwargs)
runner_log['test/mean_score']

In [ ]:
dataset_path = cfg.task.dataset.dataset_path
dataset_path

In [ ]:
h5py_file = h5py.File(dataset_path, 'r')
demo_names= list(h5py_file['data'].keys())
len(demo_names)

In [ ]:
demo_name = demo_names[0]
init_state = h5py_file['data'][demo_name]['states'][0]

In [ ]:
selected_demos = ['demo_1', 'demo_20', 'demo_33', 'demo_41', 'demo_67']
init_states =[]
for demo_name in selected_demos:
    init_state = h5py_file['data'][demo_name]['states'][0]
    init_states.append(init_state)
len(init_states), init_states[0].shape

In [ ]:
# cfg_te=cfg.task.env_runner
cfg_te={key:value for key,value in cfg.task.env_runner.items()}
cfg_te['init_states'] = init_states

In [ ]:
env_runner = hydra.utils.instantiate(
    cfg_te,
    output_dir=output_dir)

In [ ]:
kwargs={'epoch':100, 'save_rollout':True}
runner_log = env_runner.run(policy, kwargs)
runner_log['infinit/mean_score']